In [2]:
import sys
sys.path.append('/home/hice1/awagh31/scratch/miniconda3/envs/vlm-debiasing/lib/python3.12/site-packages')


In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

# Load the metadata CSV file
metadata = pd.read_csv('/home/hice1/awagh31/scratch/scripts/labels_visual_files_dn.csv')


/scratch/942792/ipykernel_1303103/1805313894.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
embeddings = []
valid_indices = []

In [5]:
#folder containing npy files
npy_base_dir = '/home/hice1/awagh31/scratch/final_embeddings_dn/features' 

In [6]:
import os
from sklearn.decomposition import PCA

In [7]:
for idx, csv_path in enumerate(metadata['file_path']):
    try:
        
        filename = os.path.basename(csv_path).replace('.csv', '')
        #print(filename)
        # Construct the expected path to the corresponding .npy file
        #change these lines if required, basically npy_path should be the path to each npy file
        npy_path = os.path.join(npy_base_dir, filename, f"{filename}.npy") 
        #print(npy_path)
        embedding = np.load(npy_path, allow_pickle=True) 
         # Assuming embeddings are stored as .npy files
        #embedding_standardized = standardize_embedding(embedding, standard_dim)
        #print(npy_path, embedding.shape)
        #embedding_reduced = pca.fit_transform(embedding.reshape(1, -1)).flatten()
        embeddings.append(embedding)
        valid_indices.append(idx)
    except (ValueError, IOError):
        print(f"Skipping {npy_path} due to loading error.")

Skipping /home/hice1/awagh31/scratch/final_embeddings_dn/features/609_densenet201/609_densenet201.npy due to loading error.


In [8]:
embeddings_modified = [[sub[1:] for sub in embedding] for embedding in embeddings]

In [9]:
X = np.vstack(embeddings_modified)

In [10]:
try:
    metadata_filtered = metadata.iloc[valid_indices]
except IndexError:
    print("Some indices in valid_indices are out-of-bounds for metadata. Check your filtering steps.")

# After filtering, extract the PTSD labels
y = metadata_filtered['PTSD_label'].values

# Verify that X and y have the same number of samples
print(f"Shape of X: {X.shape}, Shape of y: {y.shape}")
X = np.array(X)
y = np.array(y)

# Get the number of samples in each
num_samples = min(X.shape[0], y.shape[0])

# Truncate X and y to the same number of samples
X_aligned = X[:num_samples]
y_aligned = y[:num_samples]

# Now proceed with train_test_split if the shapes match
if X_aligned.shape[0] == y_aligned.shape[0]:
    X_train, X_test, y_train, y_test = train_test_split(X_aligned, y_aligned, test_size=0.2, random_state=42)
else:
    print("Mismatch in the number of samples between X and y.")

Shape of X: (260909, 1921), Shape of y: (274,)


In [11]:

if 'gender' in metadata.columns:
    le = LabelEncoder()
    metadata['gender'] = le.fit_transform(metadata['gender'])

# Split the dataset into training and testing sets (80-20 split)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=50, random_state=42)
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

In [12]:
# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.7454545454545455
Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.98      0.85        41
           1       0.50      0.07      0.12        14

    accuracy                           0.75        55
   macro avg       0.63      0.52      0.49        55
weighted avg       0.69      0.75      0.67        55



In [13]:
!pip install fairlearn

Defaulting to user installation because normal site-packages is not writeable


In [14]:
metadata.head()

,gender,split,PTSD_label,age,PTSD_severity,file_path
0,0,train,0,45,22.0,/home/hice1/awagh31/scratch/original/data_unta...
1,1,test,0,69,23.0,/home/hice1/awagh31/scratch/original/data_unta...
2,1,train,0,25,19.0,/home/hice1/awagh31/scratch/original/data_unta...
3,0,train,1,58,67.0,/home/hice1/awagh31/scratch/original/data_unta...
4,1,dev,0,33,39.0,/home/hice1/awagh31/scratch/original/data_unta...


In [16]:
from fairlearn.metrics import demographic_parity_difference

# Assuming `gender` corresponds to the original metadata
# Split the dataset into training and testing sets for `gender`
gender_train, gender_test = train_test_split(metadata['gender'].values, test_size=0.2, random_state=42)

# Ensure that `gender_test`, `y_test`, and `y_pred` are aligned
print(f"Shape of gender_test: {gender_test.shape}")
print(f"Shape of y_test: {y_test.shape}")
print(f"Shape of y_pred: {y_pred.shape}")

# Calculate Demographic Parity Difference
demographic_parity = demographic_parity_difference(
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=gender_test
)

print(f"Demographic Parity Difference: {demographic_parity}")

Shape of gender_test: (55,)
Shape of y_test: (55,)
Shape of y_pred: (55,)
Demographic Parity Difference: 0.032507739938080496


In [19]:
from fairlearn.metrics import MetricFrame, selection_rate
metric_frame = MetricFrame(metrics=selection_rate, 
                           y_true=y_test, 
                           y_pred=y_pred, 
                           sensitive_features=gender_test)

# Get selection rates for each group
selection_rates = metric_frame.by_group
print("Selection Rates by Group:")
print(selection_rates)


Selection Rates by Group:
sensitive_feature_0
0    0.058824
1    0.026316
Name: selection_rate, dtype: float64


In [23]:
group_a_rate = selection_rates[1]
group_b_rate = selection_rates[0]

if group_b_rate > 0:  # Avoid division by zero
    demographic_parity_ratio = group_a_rate / group_b_rate
    print(f"Demographic Parity Ratio: {demographic_parity_ratio}")

Demographic Parity Ratio: 0.4473684210526316
